In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

import logging
logging.getLogger().setLevel(logging.INFO)

import datetime 
import copy
import multiprocessing as mp
import time

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash

# from config import config
# from pof.loader.asset_model_loader import AssetModelLoader
# from pof import Component, FailureMode, Task
# from pof.interface.dashlogger import DashLogger
# from pof.interface.layouts import *
# from pof.interface.figures import update_condition_fig, update_cost_fig, update_pof_fig, make_inspection_interval_fig

In [2]:
external_stylesheets=[dbc.themes.BOOTSTRAP]

## Working Semaphore example

In [3]:
class Semaphore:
    def __init__(self, filename='semaphore.txt'):
        self.filename = filename
        with open(self.filename, 'w') as f:
            f.write('done')

    def lock(self):
        with open(self.filename, 'w') as f:
            f.write('working')

    def unlock(self):
        with open(self.filename, 'w') as f:
            f.write('done')

    def is_locked(self):
        return open(self.filename, 'r').read() == 'working'

In [3]:
semaphore = Semaphore()

def long_process():
    if semaphore.is_locked():
        raise Exception('Resource is locked')
    semaphore.lock()
    time.sleep(7)
    semaphore.unlock()
    return datetime.datetime.now()

app = JupyterDash(external_stylesheets=external_stylesheets)

def layout():
    return html.Div([
        html.Button('Run Process', id='button'),
        dcc.Interval(id='interval', interval=500),
        dcc.RadioItems(
            id='lock',
            options=[{'label': i, 'value': i} for i in ['Running...', 'Free']]),
        html.Div(id='output', children="no output yet")
    ])

app.layout = layout

@app.callback(
    Output('lock', 'value'),
    [Input('interval', 'n_intervals')])
def display_status(*args):
    return 'Running...' if semaphore.is_locked() else 'Free'

@app.callback(
    Output('output', 'children'),
    [Input('button', 'n_clicks')])
def run_process(*args):
    return 'Finished at {}'.format(long_process())

app.scripts.config.serve_locally = True

if __name__ == '__main__':
    app.run_server(mode='inline', port=8000)

NameError: name 'Semaphore' is not defined

## Antoehr example

In [27]:
class Sem:
    def __init__(self):
        self.n_updates = 0
        self.n_iterations = 1
        self.n = 1
        self.active = True
        self.n_max = 100

    def update(self):
        self.n_updates = self.n_updates + 1
        self.up_to_date = False

    def do_work(self):
        # Set initial
        self.active = True
        self.n = self.n_iterations

        while self.n < self.n_max and self.active is True:
            #Progress bar inputs
            if self.n == self.n_iterations:
                self.n_iterations = self.n*2
            
            # Do work
            time.sleep(1)

            self.n = self.n + 1

        return f"complete at {self.n}"

    def next_sim(self):
        self.active = True

        while self.n <= self.n_max and self.n <= self.n_iterations and self.active is True:
            # Do work
            time.sleep(1)
            self.n = self.n + 1

        self.n_iterations = self.n * 2

        return self.n_iterations

    def reset(self):
        self.n=1
        self.n_iterations= 1
        self.active = False

    def check_n(self):
        """Not needed"""
        return int(open(self.filename, 'r').read())

    def get_n(self):
        return self.n
    

In [ ]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

sem = Sem()

def layout():
    return html.Div([
        html.Button('Run Process', id='run-button'),
        html.Button('Kill Process', id='kill-button'),
        dcc.Input(id='update-input', value="Update", type='text'),
        dcc.Interval(id='interval', interval=10),
        html.Div(id='run-output'),
        html.Div(id='kill-output'),
        html.Div(["Input: ", dcc.Input(id='n-input', value=sem.n_iterations, type='number')]),
        html.Div([
            dcc.Interval(id="progress-interval", n_intervals=0, interval=500),
            dbc.Progress(id="progress"),
        ])
    ])

app.layout = layout

@app.callback(
    Output('update-input', 'children'),
    [Input('update-input', 'value')])
def change_input(value):
    # Do an update
    sem.update()

@app.callback(
    Output('kill-output', 'children'),
    [Input('kill-button', 'n_clicks')])
def kill_process(*args):
    sem.reset()

# @app.callback(
#     Output('run-output', 'children'),
#     [Input('run-button', 'n_clicks')])
# def run_process(*args):
#     return 'Finished at {}'.format(sem.do_work())

@app.callback(
    [Output("n-input", "value")],
    [Input("progress-interval", "n_intervals")],
)
def update_n_iterations(*args):
    if sem.update_to_date == False:
        sem.next_sim()
    return sem.n_iterations

@app.callback(
    [Output("progress", "value"), Output("progress", "children")],
    [Input("progress-interval", "n_intervals")],
)
def update_progress(n):
    n = sem.n
    if n is None:
        raise Exception("no process started")
    progress = int(int(n) / sem.n_iterations * 100)
    # only add text after 5% progress to ensure text isn't squashed too much
    return progress, f"{progress} %" if progress >= 5 else ""


app.scripts.config.serve_locally = False

if __name__ == '__main__':
    app.run_server(port=8056, mode='inline')

## Best example

In [47]:
class Semi:
    def __init__(self):
        self.n_updates = 0
        self.n_iterations = 1
        self.n = 1
        self.active = True
        self.up_to_date = True
        self.n_max = 100

    def update(self):
        self.n_updates = self.n_updates + 1
        self.up_to_date = False

    def next_sim(self):
        self.active = True
        self.up_to_date = True
        self.n = self.n_iterations

        while self.n < self.n_max and self.active is True and self.up_to_date is True:
            #Progress bar inputs
            if self.n == self.n_iterations:
                self.n_iterations = self.n*4
            
            # Do work
            time.sleep(1)

            self.n = self.n + 1


    def reset(self):
        self.n=1
        self.n_iterations= 1
        self.active = False

In [1]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

sem = Semi()

def layout():
    return html.Div([
        html.Button('Run Process', id='run-button'),
        html.Button('Kill Process', id='kill-button'),
        dcc.Input(id='update-input', value="Update", type='text'),
        dcc.Interval(id='interval', interval=10),
        html.Div(id='run-output'),
        html.Div(id='kill-output'),
        html.Div(["Input: ", dcc.Input(id='n-input', value=sem.n_iterations, type='number')]),
        html.Div([
            dcc.Interval(id="progress-interval", n_intervals=0, interval=500),
            dbc.Progress(id="progress"),
        ])
    ])

app.layout = layout

@app.callback(
    Output('update-input', 'children'),
    [Input('update-input', 'value')])
def change_input(value):
    # Do an update
    sem.update()

@app.callback(
    Output('kill-output', 'children'),
    [Input('kill-button', 'n_clicks')])
def kill_process(*args):
    sem.reset()


@app.callback(
    [Output("n-input", "value")],
    [Input("progress-interval", "n_intervals")],
)
def update_n_iterations(*args):
    if sem.up_to_date == False:
        sem.next_sim()
    return [sem.n_iterations]

@app.callback(
    [Output("progress", "value"), Output("progress", "children")],
    [Input("progress-interval", "n_intervals")],
)
def update_progress(n):
    n = sem.n
    if n is None:
        raise Exception("no process started")
    progress = int(int(n) / sem.n_iterations * 100)
    # only add text after 5% progress to ensure text isn't squashed too much
    return progress, f"{progress} %" if progress >= 5 else ""


app.scripts.config.serve_locally = False

if __name__ == '__main__':
    app.run_server(port=8056, mode='inline')

NameError: name 'JupyterDash' is not defined

## Looping Example

In [ ]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

run 

def layout():
    return html.Div([
        dcc.RadioItems(
            id='sim-radio',
            options=[{'label': i, 'value': i} for i in ['on', 'off']],
            value='on',
            labelStyle={'display': 'inline-block'}
        ),
        html.Div(id='sim-state'),
        html.Div(id='graph-state'),
    ])

app.layout = layout

@app.callback(
    Output('sim-output', 'children'),
    [Input('sim-radio', 'value')])
def change_input(value):
    
    global run
    if value == 'on':
        run = True
    elif value == 'off':
        run = False

@app.callback(
    Output('update-input', 'children'),
    [Input('update-input', 'value')])
def change_input(value):
    # Do an update
    sem.update()

@app.callback(
    Output('kill-output', 'children'),
    [Input('kill-button', 'n_clicks')])
def kill_process(*args):
    sem.reset()


@app.callback(
    [Output("n-input", "value")],
    [Input("progress-interval", "n_intervals")],
)
def update_n_iterations(*args):
    if sem.up_to_date == False:
        sem.next_sim()
    return [sem.n_iterations]

@app.callback(
    [Output("progress", "value"), Output("progress", "children")],
    [Input("progress-interval", "n_intervals")],
)
def update_progress(n):
    n = sem.n
    if n is None:
        raise Exception("no process started")
    progress = int(int(n) / sem.n_iterations * 100)
    # only add text after 5% progress to ensure text isn't squashed too much
    return progress, f"{progress} %" if progress >= 5 else ""


app.scripts.config.serve_locally = False

if __name__ == '__main__':
    app.run_server(port=8056, mode='inline')